In [11]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import ot
import sys
import anndata
import scanpy as sc
from scipy.sparse import issparse
from scipy.sparse import csr_matrix

In [12]:
def compare_sparse_anndata_X(adata1, adata2):
    # 检查两个矩阵是否都是稀疏矩阵
    if not (issparse(adata1.X) and issparse(adata2.X)):
        return False
    
    # 转换为 CSR 格式进行比较
    return (adata1.X != adata2.X).nnz == 0


In [2]:
adata = anndata.read_h5ad("/workspace/ImputationOT/data/adata_processed.h5ad")
print(adata)
print(adata.X)
print(adata.layers['counts'])
print(adata.var['feature_types'])

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 90261 × 14087
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id'
    uns: 'dataset_id', 'genome', 'organism'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'
<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 141495252 stored elements and shape (90261, 14087)>
  Coords	Values
  (0, 23)	2.8047749996185303
  (0, 25)	2.8047749996185303
  (0, 53)	2.8047749996185303
  (0, 61)	2.8047749996185303
  (0, 68)	14.02387523651123
  (0, 84)	2.8047749996185303
  (0, 138)	2.8047749996185303
  (0, 183)	

In [50]:
## preprocess1
adata_GEX1 = adata[:, adata.var['feature_types'] == 'GEX'].copy()
adata_ADT1 = adata[:, adata.var['feature_types'] == 'ADT'].copy()
### step 1
sc.pp.normalize_total(adata_GEX1, target_sum=1e4)
sc.pp.normalize_total(adata_ADT1, target_sum=1e4)
adata1 = adata.copy()
adata1.X[:, adata.var['feature_types'] == 'GEX'] = adata_GEX1.X
adata1.X[:, adata.var['feature_types'] == 'ADT'] = adata_ADT1.X
### step 2
sc.pp.log1p(adata1)
adata_GEX1 = adata1[:, adata.var['feature_types'] == 'GEX'].copy()
adata_ADT1 = adata1[:, adata.var['feature_types'] == 'ADT'].copy()
# # ### step 3
# adata_GEX1 = adata1[:, adata1.var['feature_types'] == 'GEX'].copy()
# adata_ADT1 = adata1[:, adata.var['feature_types'] == 'ADT'].copy()
# sc.pp.highly_variable_genes(
#     adata_GEX1,
#     n_top_genes=2000,
#     subset=True
# )
# highly_variable_genes_mask = adata_GEX1.var['highly_variable']
# adata1 = adata1[:, (adata1.var['feature_types'] == 'ADT') | highly_variable_genes_mask]
# print(adata1)
# adata1 = anndata.concat([adata_GEX1, adata_ADT1], axis=1, merge="same")
# print(adata1)

In [47]:
# ### preprocess2
# adata_GEX2 = adata[:, adata.var["feature_types"] == "GEX"].copy()
# adata_ADT2 = adata[:, adata.var["feature_types"] == "ADT"].copy()
# ### step 1
# sc.pp.normalize_total(adata_GEX2, target_sum=1e4)
# sc.pp.normalize_total(adata_ADT2, target_sum=1e4)
# ### step 2
# sc.pp.log1p(adata_GEX2)
# sc.pp.log1p(adata_ADT2)
# ### step 3
# sc.pp.highly_variable_genes(
#     adata_GEX2,
#     n_top_genes=2000,
#     subset=True
# )
adata2 = anndata.concat([adata_GEX2, adata_ADT2], axis=1, merge="same")   # X(:,1): GEX, X(:,2): ADT
print(adata2)

AnnData object with n_obs × n_vars = 90261 × 2134
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [51]:
print(compare_sparse_anndata_X(adata_GEX1, adata_GEX2))
print(compare_sparse_anndata_X(adata_ADT1, adata_ADT2))
# print(compare_sparse_anndata_X(adata1, adata2))

AttributeError: 'bool' object has no attribute 'nnz'